In [5]:
from swat import *

In [6]:
s = CAS('nvidia05.perf.sas.com',10466)

s.sessionprop.setsessopt(caslib='CASUSER',timeout=3.1536E7)

NOTE: 'CASUSER(renaza)' is now the active caslib.


+ Elapsed: 0.000363s, user: 0.000341s, mem: 0.262mb

In [7]:
s.loadactionset('tkrl')

NOTE: Added action set 'tkrl'.


[actionset]

 'tkrl'

+ Elapsed: 0.332s, user: 0.89s, sys: 0.845s, mem: 0.204mb

# States and Actions

The **state** contains 5 variables. We extracted these features from 130 possible variables according to their contribution in improving the accuracy of the logistic regression.

    _S0_: count_of_suspensions_6m (Times Suspended Last 6M)
    _S1_: tot_drpd_pr1 (Number of Dropped Calls 1 Mth Prior)
    _S2_: nbr_contacts (Number Times Customer Contacted)
    _S3_: calls_care_acct (Number Calls Care Center)
    _S4_: price_mention (Price Issues Discussed)


We consider 4 **actions** for all customers:

    a = 0: do nothing
    a = 1: if price_mention>0, then reduce the price 10 %. This will lead to 20% reduction in churn
    a = 2: if price_mention>0, then reduce the price 20 %. This will lead to 40% reduction in churn
    a = 3: if price_mention>0, then reduce the price 30 %. This will lead to 50% reduction in churn



# Environment
Using the 5 state variables above, we can train a model using logistic regression that classifies customer churn behavior accurately. 

Confusion Matrix:

    [[10857,   2],
     [    0, 453]]
 
Accuracy: 0.9998231966053748

We use the trained model in reward prediction.

**Reward**: We let the churn to have a cost equal to 12 month of "revenue per user". So, if we can save a customer from churning, we will get 12 * monthly revenue of that user. If a customer churns, we get 0 reward. Depending on action, we adjust the reward to reflect the cost of that particular action in reward.

# Train using CustomerSim environment

In [15]:
s.rlTrainDqn(
        environment=dict(type='remote', Name='ChurnSim-v2', url='10.122.32.31:15008'),
        seed = 298,
        optimizer=dict(method='ADAM', miniBatchSize=128, lr=0.001),
        numEpisodes = 10000,
        gamma = 0.,
        gpu=True,
        testInterval = 100,
        numTestEpisodes = 10000,
        nThreads=1,
        exploration=dict(type='linear', initialEpsilon=0.95, minEpsilon=0.0, explorationFraction=.8),
        targetUpdateInterval = 1,
        minReplayMemory = 1,
        maxReplayMemory = 500,
        modelOut=dict(name='dqnWeights', replace=True),
        finalTargetCopy=True,
#        _debug=dict(ranks='0', display='vmw-d10d548.na.SAS.com:0', debugger='/u/renaza/bin/start-gdb-9.1'),
        QModel=[128, 128]
    )

NOTE: Using GPU 1 on nvidia05.perf.sas.com.
         Episode=        0 AvgQValue=     0 AvgTarget=     0 AvgLoss=     0 TestReward=57.398
         Episode=      100 AvgQValue= 39.31 AvgTarget=48.204 AvgLoss=1022.6 TestReward=51.766
         Episode=      200 AvgQValue=42.535 AvgTarget=43.654 AvgLoss=749.16 TestReward=51.964
         Episode=      300 AvgQValue=48.069 AvgTarget=48.476 AvgLoss=   416 TestReward=64.814
         Episode=      400 AvgQValue= 50.04 AvgTarget=50.537 AvgLoss=2466.2 TestReward=67.205
         Episode=      500 AvgQValue=52.406 AvgTarget=52.164 AvgLoss=444.47 TestReward=63.862
         Episode=      600 AvgQValue=53.457 AvgTarget=55.247 AvgLoss=1665.6 TestReward=64.344
         Episode=      700 AvgQValue=51.638 AvgTarget=51.502 AvgLoss=1327.3 TestReward=63.843
         Episode=      800 AvgQValue=58.593 AvgTarget=60.198 AvgLoss=5133.9 TestReward= 65.45
         Episode=      900 AvgQValue=57.474 AvgTarget=57.723 AvgLoss=3981.2 TestReward= 63.79
         Episode

         Episode=     8600 AvgQValue=64.565 AvgTarget=72.044 AvgLoss=  3234 TestReward=65.236
         Episode=     8700 AvgQValue=59.554 AvgTarget= 60.52 AvgLoss=408.83 TestReward=66.533
         Episode=     8800 AvgQValue=62.177 AvgTarget=60.391 AvgLoss=751.09 TestReward=66.278
         Episode=     8900 AvgQValue=81.136 AvgTarget=82.294 AvgLoss=3223.6 TestReward=65.877
         Episode=     9000 AvgQValue= 72.56 AvgTarget= 60.66 AvgLoss=  2987 TestReward=65.171
         Episode=     9100 AvgQValue=63.877 AvgTarget=66.674 AvgLoss=  1339 TestReward=65.226
         Episode=     9200 AvgQValue=60.728 AvgTarget=60.545 AvgLoss=1401.1 TestReward=65.974
         Episode=     9300 AvgQValue=64.465 AvgTarget=62.716 AvgLoss=1405.3 TestReward=67.489
         Episode=     9400 AvgQValue=62.814 AvgTarget=67.266 AvgLoss=1260.2 TestReward=67.129
         Episode=     9500 AvgQValue=62.134 AvgTarget=66.792 AvgLoss=823.22 TestReward=66.752
         Episode=     9600 AvgQValue=75.059 AvgTarget=75.503

[ModelInfo]

                     Property Value
 0  Number of State Variables     5
 1          Number of Actions     4
 2                   Training   DQN
 3                  Optimizer  ADAM

[OptIterHistory]

 Iteration History
 
     Iteration      AvgQValue      AvgTarget      AvgLoss  Test Reward
 0           0  6.929228e-310  6.929228e-310     0.000000    57.398415
 1         100   3.931014e+01   4.820384e+01  1022.552059    51.766092
 2         200   4.253460e+01   4.365440e+01   749.164758    51.964441
 3         300   4.806928e+01   4.847553e+01   416.003786    64.814134
 4         400   5.004008e+01   5.053706e+01  2466.153491    67.205255
 5         500   5.240601e+01   5.216385e+01   444.470514    63.862179
 6         600   5.345697e+01   5.524734e+01  1665.556780    64.344088
 7         700   5.163752e+01   5.150178e+01  1327.330690    63.842516
 8         800   5.859270e+01   6.019829e+01  5133.943864    65.450255
 9         900   5.747390e+01   5.772266e+01  3981.192253    63.790080
 10       1000   6.678121e+01   5.680728e+01  5331.604877    65.007090
 11       1100   6.573803e+01   6.021780e+01  5000.602930    66.477307
 12       1200   6.534048e+01   7.425467e+01  5932.082593    64.393616
 13       1300   6.794382e+01   7.273069e+01  4546.116086    66.196225
 14       1400   5.515832e+01   4.719525e+01  2288.013017    67.885789
 15       1500   5.746619e+01   5.687623e+01  1903.233059    63.099895
 16       1600   5.437459e+01   4.827361e+01  1541.796262    65.975046
 17       1700   4.790009e+01   5.303912e+01  1316.519730    66.976690
 18       1800   5.335602e+01   5.305397e+01  4070.063520    68.356221
 19       1900   4.962161e+01   4.617300e+01   841.925878    61.972271
 20       2000   5.598248e+01   5.244420e+01  2791.493480    64.526057
 21       2100   4.471803e+01   4.823348e+01  1045.788237    59.733759
 22       2200   5.310354e+01   4.836451e+01  1140.804998    57.557154
 23       2300   5.044588e+01   5.268875e+01   591.714115    60.455851
 24       2400   4.851434e+01   5.577555e+01  3749.863599    61.565567
 25       2500   5.509188e+01   5.271615e+01   878.632486    55.305831
 26       2600   5.746353e+01   5.788533e+01  1949.029800    63.963010
 27       2700   5.253290e+01   5.664294e+01   965.883526    66.318376
 28       2800   5.972761e+01   5.119535e+01  2183.378936    64.972421
 29       2900   6.762029e+01   5.617092e+01  3500.773059    61.673972
 30       3000   6.160331e+01   6.326306e+01  4894.696554    67.274900
 31       3100   6.832103e+01   7.574638e+01  8550.442666    63.702384
 32       3200   6.112575e+01   5.362864e+01  3123.240200    65.099665
 33       3300   6.311808e+01   6.953157e+01  8896.789619    64.198645
 34       3400   5.561857e+01   4.931527e+01  2073.865720    63.771691
 35       3500   5.835583e+01   6.931918e+01  5099.401258    67.496356
 36       3600   5.232946e+01   5.669456e+01   955.584571    65.104784
 37       3700   6.000105e+01   5.844164e+01  3894.388308    64.590034
 38       3800   5.451556e+01   5.895620e+01  1639.510918    63.782548
 39       3900   6.169633e+01   5.832680e+01  2877.087718    63.297630
 40       4000   6.054852e+01   6.030830e+01  3140.463616    64.299205
 41       4100   5.814492e+01   5.097868e+01  2845.673190    66.808307
 42       4200   5.159658e+01   4.942937e+01   534.000508    66.644471
 43       4300   6.006219e+01   6.482384e+01  2988.501408    64.133528
 44       4400   5.865188e+01   5.469613e+01  1515.160549    67.480252
 45       4500   6.526726e+01   6.437731e+01  5996.965232    66.543702
 46       4600   7.261230e+01   5.707129e+01  3443.008633    62.683189
 47       4700   7.134849e+01   5.242092e+01  5295.038712    60.565690
 48       4800   7.217141e+01   8.301759e+01  8466.405904    60.294234
 49       4900   6.397798e+01   5.213804e+01  2875.591698    61.271828
 50       5000   7.080944e+01   7.593018e+01  8037.905953    62.554964
 51       5100   6.157215e+01   4.986961e+01  4128.917873    65.729280
 52 

In [19]:
s.table.save(table = "dqnWeights", name = "dqn-churn-weights.sashdat", replace = True)
s.table.attribute(task ='CONVERT', name = 'dqnWeights')
s.table.save(table = "dqnWeights.attrs", name = "dqn-churn-attrs.sashdat", replace = True)


NOTE: Cloud Analytic Services saved the file dqn-churn-weights.sashdat in caslib CASUSER(renaza).
NOTE: Cloud Analytic Services saved the file dqn-churn-attrs.sashdat in caslib CASUSER(renaza).


[caslib]

 'CASUSER(renaza)'

[name]

 'dqn-churn-attrs.sashdat'

+ Elapsed: 0.00494s, user: 0.00128s, sys: 2.9e-05s, mem: 16.6mb

# Score Using Historical Data
#### Load data: We first load historical customer interaction tuples


In [28]:
# to be able to upload the models, they should be in your local machine
s.upload("./dqn-churn-weights.sashdat", casout={"name":"dqnWeights","replace":True})
s.upload("./dqn-churn-attrs.sashdat", casout={"name":"dqnWeights-attrs","replace":True})
s.table.attribute(table='dqnWeights-attrs', task ='ADD', name = 'dqnWeights')


NOTE: Cloud Analytic Services made the uploaded file available as table DQNWEIGHTS in caslib rl_testing.
NOTE: The table DQNWEIGHTS has been created in caslib rl_testing from binary data uploaded to Cloud Analytic Services.
NOTE: Cloud Analytic Services made the uploaded file available as table DQNWEIGHTS-ATTRS in caslib rl_testing.
NOTE: The table DQNWEIGHTS-ATTRS has been created in caslib rl_testing from binary data uploaded to Cloud Analytic Services.


+ Elapsed: 0.000332s, user: 0.000307s, mem: 0.291mb

# Score using CustomerSim Environment

In [29]:
nBaselineEP = 10000 

s.rlScore(model='dqnWeights',
          environment=dict(type='remote', Name='ChurnSim-v2', url='10.122.32.31:15008'),
         casout=dict(name='scoreTableEnv', replace=True),
         numEpisodes=nBaselineEP,
         logFreq=2000,
          writeQValues=True,
         )

         Episode=     1000 Step=     1000 LastReward= 89.98 AverageReward=68.433
NOTE: Reinforcement learning rlScore action complete.


,Property,Value
0,Number of State Variables,5
1,Number of Actions,4
2,Training,DQN
3,Optimizer,ADAM


In [17]:
import pandas
pandas.set_option('display.max_rows', None)
s.fetch('scoreTableEnv', to=100)

,_Step_,_Episode_,_S0_,_S1_,_S2_,_S3_,_S4_,_Action_,_Reward_,_Done_,_QVal_0_,_QVal_1_,_QVal_2_,_QVal_3_
0,0.0,1.0,0.000000,7.0,0.0,0.0,0.0,0.0,99.574997,1.0,72.587826,20.065924,6.906663,-2.600313
1,0.0,2.0,0.000000,5.0,0.0,0.0,0.0,0.0,51.476662,1.0,67.800086,22.256415,11.222183,2.368642
2,0.0,3.0,0.276471,2.0,0.0,0.0,0.0,0.0,51.775433,1.0,58.668563,26.029938,8.939692,9.641790
3,0.0,4.0,0.276471,2.0,0.0,0.0,0.0,0.0,51.775433,1.0,58.668563,26.029938,8.939692,9.641790
4,0.0,5.0,0.000000,4.0,1.0,0.0,0.0,0.0,79.566658,1.0,69.013125,25.965453,-10.506485,9.464752
5,0.0,6.0,0.000000,1.0,0.0,0.0,0.0,0.0,58.995998,1.0,58.204208,26.734848,12.535982,10.779566
6,0.0,7.0,0.000000,21.0,0.0,0.0,0.0,0.0,126.826599,1.0,63.803991,-22.509418,-27.520971,-3.550552
7,0.0,8.0,0.000000,1.0,0.0,0.0,0.0,0.0,45.428329,1.0,58.204208,26.734848,12.535982,10.779566
8,0.0,9.0,0.000000,0.0,1.0,0.0,0.0,0.0,84.980003,1.0,62.969219,6.884247,-5.770504,-19.466065
9,0.0,10.0,5.000000,0.0,0.0,0.0,0.0,0.0,43.105549,1.0,74.922621,-39.966015,-80.864179,-154.154575


In [18]:
import numpy as np
rl_rew = s.fetch('scoreTableEnv', to=1e6, maxrows=1e6)['Fetch']['_Reward_']
rl_mean_rew = np.sum(rl_rew)/nBaselineEP
print("DQN cumulative reward is : ", rl_mean_rew)

DQN cumulative reward is :  68.32684606642276


# Selected Action for Churned Customers

In [23]:
s.addCaslib(name='rl_testing', path='/bigdisk/lax/renaza/env/gym-customersim/churn_data', dataSource=dict(srcType="DNFS"),
                    subdirectories=True)

NOTE: Cloud Analytic Services made the DNFS file looking_glass_v5.csv available as table CHURN in caslib rl_testing.


ERROR: The caslib rl_testing is a duplicate, parent or subpath of caslib rl_testing.
ERROR: Could not add caslib 'rl_testing'. Make sure that the caslib does not already exist and that you have permissions to add caslibs to Cloud Analytic Services.
ERROR: The action stopped due to errors.


,Var1,old_st,Customer_ID,upsell_xsell,churn,lifetime_value,avg_arpu_3m,acct_age,billing_cycle,nbr_contracts_ltd,...,pymts_late_ltd,calls_care_ltd,MB_Data_Usg_M04,MB_Data_Usg_M05,MB_Data_Usg_M06,MB_Data_Usg_M07,MB_Data_Usg_M08,MB_Data_Usg_M09,seconds_of_data_norm,seconds_of_data_log
0,0.0,VA,471.0,0.0,0.0,7.435320,44.980000,55.000000,4.0,7.0,...,5.0,57.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,4.110874
1,1.0,CA,1057.0,0.0,0.0,10.563168,44.989330,47.000000,3.0,5.0,...,5.0,115.0,0.0,0.0,0.0,0.0,330.0,646.0,48300.0,10.785208
2,2.0,PA,1177.0,0.0,0.0,8.617473,51.775434,46.176471,7.0,4.8,...,5.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,14700.0,9.595671
3,3.0,IL,1181.0,0.0,0.0,6.227722,51.775434,75.000000,7.0,4.8,...,5.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,4.0,CA,1185.0,1.0,0.0,9.120875,83.320000,50.000000,5.0,3.0,...,5.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,15840.0,9.670357


In [36]:
s.loadtable(caslib='rl_testing', path='looking_glass_v5_churn.csv',
         casout=dict(name='churn', replace=True), importOptions=dict(fileType='csv'))
s.fetch('churn', to=5)

NOTE: Cloud Analytic Services made the DNFS file looking_glass_v5_churn.csv available as table CHURN in caslib rl_testing.


,Customer_ID,churn,lifetime_value,avg_arpu_3m,state,city,zipcode_primary,_S0_,_S1_,_S2_,_S3_,_S4_
0,1271.0,1.0,8.675973,48.452220,OR,Powell Butte,97753.0,3.000000,11.0,1.0,3.0,2.0
1,1418.0,1.0,9.063313,114.636600,TN,Trezevant,38258.0,4.000000,7.0,0.0,3.0,1.0
2,6950.0,1.0,8.294550,62.276549,OH,West Chester,45069.0,3.047619,9.0,1.0,3.0,1.0
3,9117.0,1.0,8.967530,20.395000,ID,Coeur D Alene,83815.0,2.000000,7.0,1.0,2.0,1.0
4,9145.0,1.0,8.133861,79.990000,MA,Waltham,2452.0,2.000000,5.0,0.0,2.0,1.0


In [37]:
s.rlScore(model='dqnWeights',
          table='churn',
         casout=dict(name='scoreTableHist', replace=True),
         numEpisodes=1,
         logFreq=1,
          writeQValues=True,
         )

NOTE: Reinforcement learning rlScore action complete.


,Property,Value
0,Number of State Variables,5
1,Number of Actions,4
2,Training,DQN
3,Optimizer,ADAM
,Description,Value
0,Number of Observations Read,2354
1,Number of Observations Used,2198


In [39]:
s.fetch('scoreTableHist', to=5)

,_Action_,_QVal_0_,_QVal_1_,_QVal_2_,_QVal_3_
0,3.0,-66.189304,170.413986,57.363883,612.079215
1,3.0,-30.102187,92.867774,18.446329,377.529701
2,3.0,-15.812680,118.380679,20.419993,405.044282
3,3.0,4.753653,110.901229,17.781216,337.937053
4,3.0,-11.303078,84.228068,20.126333,299.916900
